In [ ]:
import pygame
import random
pygame.init()

WIDTH, HEIGHT = 400, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Racer Game")


BLUE  = (0, 0, 255)
RED   = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)


speed = 12  
score = 0  
speed_increment_threshold = 7  
speed_increase = 0.2  
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, BLACK)


player_image = pygame.image.load("Player.png")
player_image = pygame.transform.scale(player_image, (50, 100))
enemy_image = pygame.image.load("Enemy.png")
enemy_image = pygame.transform.scale(enemy_image, (50, 100))
coin_image = pygame.image.load("coin.gif")
coin_image = pygame.transform.scale(coin_image, (30, 30))
background_image = pygame.image.load("AnimatedStreet.png")
background_image = pygame.transform.scale(background_image, (WIDTH, HEIGHT))

crash_sound = pygame.mixer.Sound("crash.wav")
background_sound = pygame.mixer.Sound("background.wav")
background_sound.play(-1)

class Car:
    def __init__(self, x, y, image):
        self.image = pygame.transform.scale(image, (50, 100))
        self.rect = self.image.get_rect(topleft=(x, y))

    def draw(self):
        screen.blit(self.image, self.rect)

class Player(Car):
    def move(self, keys):
        if keys[pygame.K_LEFT] and self.rect.left > 50:
            self.rect.x -= 10
        if keys[pygame.K_RIGHT] and self.rect.right < WIDTH - 50:
            self.rect.x += 10

class Enemy(Car):
    def update(self):
        global speed 
        self.rect.y += speed
        if self.rect.top > HEIGHT: 
            self.rect.y = -100
            self.rect.x = random.choice([75, 175, 275])

class Coin:
    def __init__(self):
        self.image = pygame.transform.scale(coin_image, (30, 30))
        self.radius = 15
        self.value = random.randint(1, 2)
        self.rect = pygame.Rect(random.choice([75, 175, 275]), -100, self.radius * 2, self.radius * 2)

    def update(self):
        self.rect.y += speed  
        if self.rect.top > HEIGHT:  
            self.reset()

    def draw(self):
        screen.blit(self.image, self.rect)

    def reset(self):
        self.rect.y = -100
        self.rect.x = random.choice([75, 175, 275])
        self.value = random.randint(1, 2) 

player = Player(WIDTH // 2 - 25, HEIGHT - 120, player_image)
enemies = [Enemy(random.choice([75, 175, 275]), -100, enemy_image)]
coins = [Coin()]

running = True
clock = pygame.time.Clock()

while running:
    screen.blit(background_image, (0, 0))  

    keys = pygame.key.get_pressed() 
    player.move(keys) 

    for enemy in enemies:
        enemy.update()
        enemy.draw()
        if player.rect.colliderect(enemy.rect):
            crash_sound.play()
            pygame.time.delay(1000)  
            screen.fill(RED)
            screen.blit(game_over, (30, 250))
            pygame.display.update()
            pygame.time.delay(2000)
            running = False

    for coin in coins:
        coin.update()
        coin.draw()
        if player.rect.colliderect(coin.rect): 
            score += coin.value  
            coin.reset() 
    if score % speed_increment_threshold == 0 and score > 0:
        speed += speed_increase 
    player.draw()  

    # счет
    score_text = font_small.render(f"Score: {score}", True, WHITE)
    screen.blit(score_text, (10, 10))

  
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    pygame.display.flip()  
    clock.tick(30)  

pygame.quit()


In [15]:
import pygame
import random

pygame.init()

WIDTH, HEIGHT = 600, 400
CELL_SIZE = 20
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

BROWN = (89, 45, 0)
PINK = (255, 192, 203)
GREEN = (34, 139, 34)

snake = [(100, 100), (90, 100), (80, 100)]
direction = (CELL_SIZE, 0)

class Food:
    def __init__(self):
        self.position = self.generate_position()
        self.weight = random.choice([1, 2]) 
        self.timer = random.randint(50, 150)  
    
    def generate_position(self):
        while True:
            x = random.randint(0, (WIDTH // CELL_SIZE) - 1) * CELL_SIZE
            y = random.randint(0, (HEIGHT // CELL_SIZE) - 1) * CELL_SIZE
            if (x, y) not in snake:
                return x, y
    
    def update(self):
        self.timer -= 1
        if self.timer <= 0:
            self.position = self.generate_position()
            self.weight = random.choice([1, 2])
            self.timer = random.randint(50, 150)
    
    def draw(self):
        pygame.draw.rect(screen, GREEN, (*self.position, CELL_SIZE, CELL_SIZE))

food = Food()
score = 0
level = 1
speed = 10

running = True
while running:
    screen.fill(BROWN)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP and direction != (0, CELL_SIZE):
                direction = (0, -CELL_SIZE)
            elif event.key == pygame.K_DOWN and direction != (0, -CELL_SIZE):
                direction = (0, CELL_SIZE)
            elif event.key == pygame.K_LEFT and direction != (CELL_SIZE, 0):
                direction = (-CELL_SIZE, 0)
            elif event.key == pygame.K_RIGHT and direction != (-CELL_SIZE, 0):
                direction = (CELL_SIZE, 0)

    new_head = (snake[0][0] + direction[0], snake[0][1] + direction[1])
    
    if (new_head[0] < 0 or new_head[0] >= WIDTH or
        new_head[1] < 0 or new_head[1] >= HEIGHT or
        new_head in snake):
        running = False
    
    snake.insert(0, new_head)
    
    if new_head == food.position:
        score += food.weight  
        food = Food() 
        if score % 3 == 0:
            level += 1
            speed += 2
    else:
        snake.pop()
    
    food.update()
    food.draw()
    
    for segment in snake:
        pygame.draw.rect(screen, PINK, (*segment, CELL_SIZE, CELL_SIZE))
    
    font = pygame.font.SysFont(None, 30)
    score_text = font.render(f"Score: {score}  Level: {level}", True, PINK)
    screen.blit(score_text, (10, 10))
    
    pygame.display.flip()       
    clock.tick(speed)

pygame.quit()


In [16]:
import pygame
import sys

pygame.init()

WIDTH, HEIGHT = 800, 600
WHITE = (255, 255, 255)
COLORS = [
    (0, 0, 0), (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0),
    (255, 165, 0), (128, 0, 128), (0, 255, 255), (255, 192, 203), (128, 128, 128)
]

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pygame Paint")

current_color = COLORS[0]
drawing = False
shape = "brush"  
start_pos = None
canvas = pygame.Surface((WIDTH, HEIGHT))
canvas.fill(WHITE)

try:
    eraser_img = pygame.image.load('Eraser.png').convert_alpha()
    eraser_img = pygame.transform.scale(eraser_img, (30, 30))  
except:
    eraser_img = pygame.Surface((30, 30), pygame.SRCALPHA)
    pygame.draw.rect(eraser_img, (255, 255, 255, 255), (0, 0, 30, 30))

def draw_color_buttons():
    for i, color in enumerate(COLORS):
        pygame.draw.rect(screen, color, (10 + i * 40, 10, 30, 30))
    screen.blit(eraser_img, (10 + len(COLORS) * 40, 10))

def draw_shape(start, end, shape, color, surface):
    x1, y1 = start
    x2, y2 = end
    
    if shape == "rectangle" or shape == "square":
        size = min(abs(x2 - x1), abs(y2 - y1)) if shape == "square" else (abs(x2 - x1), abs(y2 - y1))
        pygame.draw.rect(surface, color, (min(x1, x2), min(y1, y2), size if shape == "square" else size[0], size if shape == "square" else size[1]))
    elif shape == "triangle":
        mid_x = (x1 + x2) // 2
        points = [(x1, y2), (x2, y2), (mid_x, y1)]
        pygame.draw.polygon(surface, color, points)
    elif shape == "right_triangle":
        points = [(x1, y1), (x1, y2), (x2, y2)]
        pygame.draw.polygon(surface, color, points)
    elif shape == "equilateral_triangle":
        mid_x = (x1 + x2) // 2
        height = abs(y2 - y1)
        points = [(x1, y2), (x2, y2), (mid_x, y2 - height)]
        pygame.draw.polygon(surface, color, points)
    elif shape == "rhombus":
        mid_x = (x1 + x2) // 2
        mid_y = (y1 + y2) // 2
        points = [(mid_x, y1), (x2, mid_y), (mid_x, y2), (x1, mid_y)]
        pygame.draw.polygon(surface, color, points)

running = True
while running:
    screen.fill(WHITE)
    screen.blit(canvas, (0, 0))
    draw_color_buttons()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        
        if event.type == pygame.MOUSEBUTTONDOWN:
            x, y = event.pos
            if y <= 40:
                for i in range(len(COLORS)):
                    if 10 + i * 40 <= x <= 40 + i * 40:
                        current_color = COLORS[i]
                if 10 + len(COLORS) * 40 <= x <= 40 + len(COLORS) * 40:
                    shape = "eraser"
            else:
                drawing = True
                start_pos = event.pos
        
        if event.type == pygame.MOUSEBUTTONUP:
            if shape in ["rectangle", "triangle", "square", "right_triangle", "equilateral_triangle", "rhombus"] and start_pos:
                end_pos = event.pos
                draw_shape(start_pos, end_pos, shape, current_color, canvas)
            drawing = False 
        
        if event.type == pygame.MOUSEMOTION and drawing:
            if shape == "brush":
                pygame.draw.circle(canvas, current_color, event.pos, 3)
            elif shape == "eraser":
                pygame.draw.circle(canvas, WHITE, event.pos, 15)
        
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                shape = "rectangle"
            elif event.key == pygame.K_s:
                shape = "square"
            elif event.key == pygame.K_t:
                shape = "triangle"
            elif event.key == pygame.K_y:
                shape = "right_triangle"
            elif event.key == pygame.K_e:
                shape = "equilateral_triangle"
            elif event.key == pygame.K_d:
                shape = "rhombus"
            elif event.key == pygame.K_b:
                shape = "brush"
            elif event.key == pygame.K_e:
                shape = "eraser"
    
    font = pygame.font.SysFont(None, 24)
    tool_text = f"Tool: {shape}"
    text_surface = font.render(tool_text, True, (0, 0, 0))
    screen.blit(text_surface, (10, HEIGHT - 30))
    
    pygame.display.flip()
    
pygame.quit()
sys.exit()

SystemExit: 